In [ ]:
import os
import sys
parent = os.path.abspath(os.path.join('./', os.pardir))
sys.path.insert(0, parent)

from nbp_helper_funs import *
from matplotlib import pyplot as plt
% matplotlib inline

from itertools import product
import os
import random
import string

In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
DIR = os.path.join(parent,'hardcoded plates')
im_dirs = list(os.path.join(DIR, file) for file in os.listdir(DIR) if file.endswith('.jpg'))
ims = list(openGrayScale(im) for im in im_dirs)    # removed cropContour
ims = list(cropContourImg(im, pad=20) for im in ims)
ims = list(straightenImage(im, getCorners(im)) for im in ims)
ims = list(im.filter(ImageFilter.MedianFilter(5)) for im in ims)
ims = list(ImageOps.autocontrast(im, 2) for im in ims)

im_dirs
len(ims)

In [ ]:
def makeRandomPlateString():

    plate = ''.join(random.choices(string.ascii_uppercase, k=2)) + ''.join(random.choices(string.digits, k=2)) + ' '
    plate += ''.join(random.choices(string.ascii_uppercase, k=3))
    
    return plate

print(makeRandomPlateString())

In [ ]:
# plates = loadFromPickle(os.path.join(parent,'test subset','data.pkl'))
names = list(os.path.split(im)[1] for im in im_dirs[:])
answers = list(name[2:-4].replace('-', ' ') for name in names)

print(answers)
print(len(answers))

# make plates
plates = []
for i in range(90):
    plates.append(makeRandomPlateString())


guesses = list(answer for answer in answers) 
print(guesses)

guesses.extend(list(plate for plate in plates))
    
random.shuffle(guesses)

# print(plates)
print(guesses)

In [ ]:
# print(searchDictionary(plates, 'HR25 ZRC'))
height = int(60*45/71)

getPlate(guesses[0], height)

In [ ]:
def makeMotionKernel(size=20, orientation=True):
    """Makes Numpy array for the motion blur kernel
    """
    if int(size/2) == size/2:
        size -= 1
    
    kernel_motion_blur = np.zeros((size, size))
    kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
    
    if orientation == True:
        kernel_motion_blur = kernel_motion_blur.T / size
        
    return kernel_motion_blur

plt.imshow(makeMotionKernel(), cmap='gray')
plt.show()

In [ ]:
height = int(60*45/71)
guessPlates = []
guessPlates2 = []

kernel_size = 15
# generating the kernel
kernel = makeMotionKernel(size=kernel_size)

skip = 3

for guess in guesses: guessPlates.append(convolveImages(pil2np(getPlate(guess, height)), kernel=kernel)) # getPlate
for guess in guesses: guessPlates2.append(pil2np(getPlate(guess,height)))       # getPlate2
    
# Initialise containers
searchResults, searchResults2 = {}, {}
searchResults['scores'] , searchResults['confidence'] , searchResults['answer']  = [], [], []
searchResults2['scores'], searchResults2['confidence'], searchResults2['answer'] = [], [], []

# Loop though the images
for im, answer in zip(ims, answers):
    im = pil2np(im)
    
    # Search though guessPlates
    scores, confidence   = RMSMultiSearch(im, guessPlates, skip=skip, names=guesses)
    searchResults['scores'].append(scores)
    searchResults['confidence'].append(confidence)
    searchResults['answer'].append(answer)
    
    # Search though guessPlates2
    scores2, confidence2   = RMSMultiSearch(im, guessPlates2, skip=skip, names=guesses)
    searchResults2['scores'].append(scores2)
    searchResults2['confidence'].append(confidence2)
    searchResults2['answer'].append(answer)

In [ ]:
print('              |     SCORE    CONF.    PLATE       SCORE    CONF.   PLATE        SCORE    CONF.   PLATE')
print('-'*105)
for i,im in enumerate(ims):

    # Unpack containers
    s = searchResults['scores'][i]
    c = searchResults['confidence'][i]*100
    a = searchResults['answer'][i]
    
    s2 = searchResults2['scores'][i]
    c2 = searchResults2['confidence'][i]*100
    a2 = searchResults2['answer'][i]
    
    
    color = bcolors.OKGREEN if a==s[0][1] else bcolors.FAIL
    color2 = bcolors.OKGREEN if a2==s2[0][1] else bcolors.FAIL
    line = '#1: {:s}  |  1: {:06.3f}  {:06.3f}  {:s}   2: {:06.3f}  {:06.3f}  {:s}   3: {:06.3f}  {:06.3f}, {:s}'.format(a,s[0][0],c[0],s[0][1],s[1][0],c[1],s[1][1],s[2][0],c[3],s[2][1])
    line = color + line + bcolors.ENDC
    line2 = '#2: {:s}  |  1: {:06.3f}  {:06.3f}  {:s}   2: {:06.3f}  {:06.3f}  {:s}   3: {:06.3f}  {:06.3f}, {:s}'.format('        ',s2[0][0],c2[0],s2[0][1],s2[1][0],c2[1],s2[1][1],s2[2][0],c2[3],s2[2][1])
    line2 = color + line2 + bcolors.ENDC
    print(line)
    print(line2)
    print('-'*105)

In [ ]:
for i,im in enumerate(ims):
    # Unpack containers
    s = searchResults['scores'][i]
    c = searchResults['confidence'][i]*100
    a = searchResults['answer'][i]
    
    cmap = plt.cm.Greens_r if a==s[0][1] else plt.cm.Reds_r
    plt.imshow(pil2np(ims[i]), cmap=cmap)
    plt.title('{:s}  |  1: {:05.2f}, {:s}  Confidence: {:1.3f}'.format(a,s[0][0],s[0][1], c[0]))
    plt.show()

In [ ]:
os.path.split(im_dirs[0])[1]

In [ ]:
getPlate('adfafd', height=60)